# Adv_seq2seq

 - Encoder는 bidirectional, decoder는 tf.nn.raw_rnn

In [5]:
import numpy as np
import tensorflow as tf
from lib import helpers
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [6]:
PAD = 0
EOS = 1

vocab_size = 10
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units * 2

encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

# Encoder

In [7]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

#Bidirectional RNN
encoder_cell = LSTMCell(encoder_hidden_units)

((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state,
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.float32, time_major=True)
    )

In [10]:
print(encoder_fw_outputs)
print(encoder_bw_outputs)
print(encoder_fw_final_state)
print(encoder_bw_final_state)

Tensor("bidirectional_rnn/fw/fw/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, ?, 20), dtype=float32)
Tensor("ReverseSequence:0", shape=(?, ?, 20), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 20) dtype=float32>)
LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 20) dtype=float32>)


In [11]:
#Concat Forward and Backward output and state, output을 사용하지 않을 경우가 아니면 attention에 쓰이게 될 것임

encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

# Decoder

In [19]:
decoder_cell = LSTMCell(decoder_hidden_units)

#time and batch s are dynamic
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [22]:
print(decoder_cell)
print(encoder_max_time, batch_size)

Tensor("unstack_1:0", shape=(), dtype=int32) Tensor("unstack_1:1", shape=(), dtype=int32)
